In [1]:
%pwd

'd:\\sign_language_conversion\\research'

In [2]:
import os 
os.chdir("../")

In [3]:
%pwd

'd:\\sign_language_conversion'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Preparedataset:
    root_dir: Path
    dataset: Path

In [5]:
from sign_language_conversion.constants import *
from sign_language_conversion.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_dataset(self) -> Preparedataset:
        config = self.config.prepare_dataset

        create_directories([config.root_dir])

        prepare_dataset_pickle = Preparedataset(
            root_dir=Path(config.root_dir),
            dataset=Path(config.dataset_path)  # Assuming 'dataset_path' is correct
        )

        return prepare_dataset_pickle

In [7]:
import os
import pickle
import mediapipe as mp
import cv2

In [8]:
class Prepare_dataset:
    def __init__(self, config: Preparedataset):
        self.config = config

    @staticmethod
    def _prepare_full_model(data_dir):
        mp_hands = mp.solutions.hands

        hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

        data = []
        labels = []
        for dir_ in os.listdir(data_dir):
            for img_path in os.listdir(os.path.join(data_dir, dir_)):
                data_aux = []

                x_ = []
                y_ = []

                img = cv2.imread(os.path.join(data_dir, dir_, img_path))
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                results = hands.process(img_rgb)
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        for i in range(len(hand_landmarks.landmark)):
                            x = hand_landmarks.landmark[i].x
                            y = hand_landmarks.landmark[i].y

                            x_.append(x)
                            y_.append(y)

                        for i in range(len(hand_landmarks.landmark)):
                            x = hand_landmarks.landmark[i].x
                            y = hand_landmarks.landmark[i].y
                            data_aux.append(x - min(x_))
                            data_aux.append(y - min(y_))

                    data.append(data_aux)
                    labels.append(dir_)

        return {'data': data, 'labels': labels}

    def save_data(self):
        data = self._prepare_full_model(self.config.dataset)

        with open(self.config.root_dir / 'data.pickle', 'wb') as f:
            pickle.dump(data, f)

In [9]:
try:
    config_manager = ConfigurationManager()
    prepare_dataset_config = config_manager.get_prepare_dataset()
    prepare_dataset = Prepare_dataset(config=prepare_dataset_config)
    prepare_dataset.save_data()
except Exception as e:
    raise e

[2024-05-25 14:32:34,728: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-25 14:32:34,732: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-25 14:32:34,734: INFO: common: created directory at: artifacts]
[2024-05-25 14:32:34,735: INFO: common: created directory at: artifacts/prepare_dataset/root_dir]


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
